# 패키지 임포트

In [3]:
import re
import pickle
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import time
import sqlite3
import numpy as np
import datetime
from datetime import datetime
from datetime import date, timedelta
from collections import Counter
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter
from ckonlpy.utils import load_ngram
from math import log10
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import openpyxl as oxl
from openpyxl import load_workbook
import ast
import warnings
import os

ModuleNotFoundError: No module named 'ckonlpy'

# 시간 설정

In [ ]:
### 금일자 기준으로 출력 아닐시
year = 2021
month = 8
day = 13
today = date(year, month, day)

### 금일자 기준으로 출력시
#today = date.today()

yesterday = today - timedelta(1)
mm = '_' + str(today)[5:7] + str(today)[8:10]
td=today.strftime('%Y%m%d')
yd=yesterday.strftime('%Y%m%d')



### 저장경로 설정
base_dir = 'C:\Users\book\Desktop\python' #원하시는 폴더경로를 설정해주시기 바랍니다.
#'/Users/new/Desktop/DC_210613'
#'C:\JBWRC_DEV'



In [ ]:
today1 = today - timedelta(1)
mm1 = '_' + str(today1)[5:7] + str(today1)[8:10]

today2 = today - timedelta(2)
mm2 = '_' + str(today2)[5:7] + str(today2)[8:10]

today3 = today - timedelta(3)
mm3 = '_' + str(today3)[5:7] + str(today3)[8:10]

today4 = today - timedelta(4)
mm4 = '_' + str(today4)[5:7] + str(today4)[8:10]



# 영역(도메인), 세부영역(서브도메인) url 확보

In [ ]:
# 1. 도메인정보 확보

"""
각 섹터별 뉴스페이지들이 존재하는 베이스정보를 확보합니다.
"""  
headers = {'User-Agent':'Mozilla/5.0','referer':'http://naver.com'}


# 메인 url 설정을 하고 해당 내용을 가져옵니다.     
page_url = "https://news.naver.com/"        
page = requests.get(page_url, headers = headers)    
soup = BeautifulSoup(page.content.decode('euc-kr','replace'),'html.parser') 

# 가져온 결과를 저장할 딕셔너리를 셋팅하고 
result = {'DOMAIN':[], 'SUB_DOMAIN':[], 'BASE_URL':[]}

# 섹션 네임을 정제하고 
domain_info_list = soup.find_all('div','lnb_inner')[0].find_all('a')

# 각 섹션(2~7은 정치 경제 사회 생활 세계 IT 입니다)
for domain_info in domain_info_list[2:8]:
    domain_name = domain_info.get_text().strip()

    # 해당 섹션 상세페이지로 접속해서 해당 페이지 정보를 가져옵니다.
    domain_url = page_url + domain_info.get('href')
    page = requests.get(domain_url, headers = headers)
    soup = BeautifulSoup(page.content.decode('euc-kr','replace'),'html.parser')

    # 섹션별 서브섹터 별로 다시 페이지 접속해서 정보를 가져옵니다
    sub_domain_info_list = soup.find_all('ul', 'nav')[0].find_all('a')
    for sub_domain_info in sub_domain_info_list[1:-2]:
        sub_domain_name = sub_domain_info.get_text().strip()
        sub_domain_url = page_url + sub_domain_info.get('href')
        result['DOMAIN'].append(domain_name)
        result['SUB_DOMAIN'].append(sub_domain_name)
        result['BASE_URL'].append(sub_domain_url)
domain_info = pd.DataFrame(result)
#domain_info.to_excel(excel_dir_domain,index=False)
print(">>>> Step 1. 도메인 정보를 확보합니다. domain 파일 생성(갱신)")
domain_info



#도메인 정보 확인
domain_list = domain_info 
domain_list

# 마지막 페이지 함수 설정(크롤링에 활용)

In [4]:
#위 각 sub domain단별로 last page 바로 적용할 함수

def get_last_page(base_url, date):
    """
    파이썬 함수 앞에 _ 가 붙는 경우 외부호출을 안하는 대상으로 규약이 있지만..
    우선 무시하세요. 클래스에서 컨버젼하느라 좀 수정된 부분이 있습니다.  
    
    영역별 날짜별로 조회시 몇페이지까지 기사가 존재하는지 체크하는 함수         
    * params 
    base_url : 세부주제영역까지 확정된 url
    date : 추출 진행할 날짜 
    
    * return
    마지막 페이지 int
    """
    date_url = base_url + '&date={}'.format(date)
    headers = {'User-Agent':'Mozilla/5.0','referer':'http://naver.com'}
    # 남은 페이지가 있다고 가정을 하고, 더 이상 페이지가 없을떄까지 반복합니다
    page_remain_yn = True
    page = 1
    while page_remain_yn:
        date_page_url = date_url + '&page={}'.format(page)
        date_page = requests.get(date_page_url, headers = headers)
        soup = BeautifulSoup(date_page.content.decode('euc-kr','replace'),'html.parser')
        try:
            last_item = soup.find_all('div','paging')[0].find_all('a')[-1]
            last_page = last_item.text
        except:
            last_item = 1
            last_page = 1
        if last_page == '다음':
            page += 10
        elif last_page == '이전':
            page += 1
            page_remain_yn = False
            return int(page)
        else:
            page_remain_yn = False
            return int(last_page)

# 데이터 전처리에 필요한 함수 만들기

In [5]:
#분석대상 판단 함수

def target_article_detect(main_title, article):
    '''
    기사 제목의 내용에 따라 분석대상이면 True를 반환 
    분석대상에서 제외되면 False 를 반환 
    1. 제목에 [---] 형태가 있는 경우 
    2. 본문이 공란인 경우
    '''
    if not re.sub('\s', '', article):
        return False
    if re.findall('\[[^\s]+\]',main_title):
        return False
    return True

#기사시간 정규표현식 함수

def is_afternoon(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return result


#기사시간 조정 함수
#20210617 오후 06 00 -> 202106171800

def only_date_time(w_time):
    num = re.findall(r"\d+", w_time) #숫자만
    if is_afternoon(w_time) == ' 오후 ':
        num2=str(int(num[3])+12)
        num[3]=num2
        return "".join([str(_) for _ in num])
    elif (is_afternoon(w_time) == ' 오전 ') & (str(num[3]) == '12'):
        num[3]='00'
        return "".join([str(_) for _ in num])
    elif (is_afternoon(w_time) == ' 오전 ') & (int(num[3]) < 12) :
        num2=num[3].zfill(2)
        num[3]=num2
        return "".join([str(_) for _ in num])
    else : 
        return "".join([str(_) for _ in num])


# 크롤링 > 1. url확보

In [6]:
#url 확보

"""
본문을 확보하기 전 대상 URL 을 먼저 수집진행하고 PROC_DVCD = '1' 로 셋팅한다. 
params 
domain : 정치, 사회, 경제, 생활/문화, 세계, IT/과학 총 6가지 영역
date : url 추출 진행할 게시날짜 

return 
url 수집결과를 반환하지만, 사용되지는 않는다. 검증용 
"""
headers = {'User-Agent':'Mozilla/5.0','referer':'http://naver.com'}
#domain_info = pd.read_excel(excel_dir_domain)
domain_list = pd.DataFrame(domain_list)
domain_info_list=domain_list.drop_duplicates(['DOMAIN'], keep='first')
list1=domain_info_list['DOMAIN'].tolist()
time_subject =[yd,td]
url_data = {'DOMAIN' :[], 'SUB_DOMAIN' :[], 'DATE':[], 'URL':[], 'PROC_DVCD':[], 'TITLE':[], 'ARTICLE':[], 'PRESS' :[], 'W_TIME':[], 'WC_VECTOR' : [], 'NOUN_VECTOR':[]}              
i=0


start_time = time.time()
for date in time_subject:
    for i in range(len(list1)):        
        sub_domain_info_list = domain_info[domain_info['DOMAIN'] ==  list1[i]]
        for idx, row in sub_domain_info_list.iterrows():
            domain = row['DOMAIN']
            sub_domain = row['SUB_DOMAIN']
            base_url = row['BASE_URL']
            last_page = get_last_page(base_url, date)
            for page in range(1, last_page+1): # 실제 마지막 페이지 숫자는 strong 형태로 되어 있어 카운트에서 빠진다.
                date_page_url = base_url + '&date={}&page={}'.format(date, page)
                date_page = requests.get(date_page_url,headers = headers) #헤더추가
                soup = BeautifulSoup(date_page.content.decode('euc-kr','replace'), 'html.parser')
                article_list = soup.find_all('ul', 'type06_headline')[0].find_all('a')
                for j in range(0,len(article_list),2): # url 1개당 2개의 'a'에 매핑되어 중복 발생. 
                    article = article_list[j]
                    url = article.get('href')
                    url_data['DOMAIN'].append(domain)
                    url_data['SUB_DOMAIN'].append(sub_domain)
                    url_data['DATE'].append(date)
                    url_data['URL'].append(url)
                    url_data['PROC_DVCD'].append('1')
                    url_data['TITLE'].append('')
                    url_data['ARTICLE'].append('')
                    url_data['PRESS'].append('')
                    url_data['W_TIME'].append('')
                    url_data['WC_VECTOR'].append('')
                    url_data['NOUN_VECTOR'].append('')
            print(">>>> Step 2. {} 영역 {} 주제 {} 기사 URL을 수집완료했습니다. ({} sec)".format(domain, sub_domain, date, round(time.time() - start_time,2)))
        i+=1
url_result = pd.DataFrame(url_data)        
url_result


NameError: name 'domain_list' is not defined

# 크롤링 > 2. 본문 확보

In [8]:
# 분석대상 판단 및 본문확보
# test : 10 ~ 14


"""
URL리스트 중 PROC_DVCD = '1'(URL만 확보) 대상을 불러와서 분석대상인지 아닌지 판단 후 
분석대상일 경우 PROC_DVCD = '2'(RAW기사 확보) 로 기사 부분을 적재한다.
분석대상이 아닌 경우 PROC_DVCD = '0' 으로 두고 기사 내용은 '' 으로 대체한다.
"""

headers = {'User-Agent':'Mozilla/5.0','referer':'http://naver.com'}

#article_data = pd.read_excel(excel_dir_article) # Domain 공란 / date 공란 / proc_dvcd = 1 조건
article_data = url_result[(url_result['PROC_DVCD']== '1')]
article_result = pd.DataFrame(article_data)
if len(article_result.index) == 0:
    print("위 조건으로 article 데이터를 먼저 수집하여 주세요")
else:
    start_time = time.time()
    for idx, row in article_data.iterrows():
        article_url = row['URL']
        domain = row['DOMAIN']
        sub_domain = row['SUB_DOMAIN']
        date = row['DATE']

        page = requests.get(article_url,headers=headers)
        soup = BeautifulSoup(page.content.decode('euc-kr','replace'), 'html.parser')

        try:
            title = soup.find('div','article_info').find('h3').get_text()
        except:
            title = '[제목없음]'
        row['TITLE'] = title

        try:
            article_ = soup.find('div','_article_body_contents').get_text()
            if 'function _flash_removeCallback' in article_ :
                article = re.split('function _flash_removeCallback\(\) \{\}',article_)[1]
            else :
                article = soup.find('div','_article_body_contents').get_text()
        except:
            article = ''
        row['ARTICLE'] = article

        try:
            press = soup.find('div', 'press_logo').find('img').get('alt')
        except:
            press = ''
        row['PRESS'] = press

        try:
            #w_time = soup.find('div','article_info').find('span').get_text()
            w_time = soup.find('span','t11').get_text()
            is_afternoon(w_time)
            w_time = only_date_time(w_time)
        except:
            w_time = ''
        row['W_TIME'] = w_time

        if target_article_detect(title, article):
            row['PROC_DVCD'] = '2'
        else:
            row['PROC_DVCD'] = '0'
        result = pd.DataFrame([row])
        article_result= pd.concat([article_result, result])
        print('>>>> Step 3. {} 영역 {} 주제  {} 일자 {}/{} 본문 확보완료 ({} sec)'.format(domain, sub_domain, date, idx + 1, len(article_data), round(time.time() - start_time, 2)))
    article_result=article_result.drop_duplicates(["DATE","TITLE"], keep="last")
#article_result.to_excel(excel_dir_navernews,index = False) 
article_result


>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 1/7507 본문 확보완료 (0.5 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 2/7507 본문 확보완료 (1.19 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 3/7507 본문 확보완료 (1.59 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 4/7507 본문 확보완료 (1.93 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 5/7507 본문 확보완료 (2.32 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 6/7507 본문 확보완료 (2.69 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 7/7507 본문 확보완료 (3.01 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 8/7507 본문 확보완료 (3.27 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 9/7507 본문 확보완료 (3.61 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 10/7507 본문 확보완료 (3.93 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 11/7507 본문 확보완료 (4.3 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 12/7507 본문 확보완료 (4.77 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 13/7507 본문 확보완료 (5.3 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 14/7507 본문 확보완료 (5.61 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 15/7507 본문 확보완료 (5.

>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 121/7507 본문 확보완료 (45.58 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 122/7507 본문 확보완료 (45.94 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 123/7507 본문 확보완료 (46.22 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 124/7507 본문 확보완료 (46.57 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 125/7507 본문 확보완료 (46.96 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 126/7507 본문 확보완료 (47.23 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 127/7507 본문 확보완료 (47.63 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 128/7507 본문 확보완료 (48.02 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 129/7507 본문 확보완료 (48.36 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 130/7507 본문 확보완료 (48.79 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 131/7507 본문 확보완료 (49.13 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 132/7507 본문 확보완료 (49.54 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 133/7507 본문 확보완료 (49.89 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 134/7507 본문 확보완료 (50.28 sec)
>>>> Step 3. 정치 영역 국

>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 239/7507 본문 확보완료 (93.91 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 240/7507 본문 확보완료 (94.29 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 241/7507 본문 확보완료 (94.6 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 242/7507 본문 확보완료 (94.99 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 243/7507 본문 확보완료 (95.42 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 244/7507 본문 확보완료 (95.79 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 245/7507 본문 확보완료 (96.3 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 246/7507 본문 확보완료 (96.63 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 247/7507 본문 확보완료 (96.94 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 248/7507 본문 확보완료 (97.58 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 249/7507 본문 확보완료 (97.99 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 250/7507 본문 확보완료 (98.34 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 251/7507 본문 확보완료 (99.06 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 252/7507 본문 확보완료 (99.4 sec)
>>>> Step 3. 정치 영역 국회/정

>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 355/7507 본문 확보완료 (141.69 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 356/7507 본문 확보완료 (142.0 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 357/7507 본문 확보완료 (142.35 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 358/7507 본문 확보완료 (142.89 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 359/7507 본문 확보완료 (143.23 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 360/7507 본문 확보완료 (143.56 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 361/7507 본문 확보완료 (143.91 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 362/7507 본문 확보완료 (144.25 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 363/7507 본문 확보완료 (144.6 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 364/7507 본문 확보완료 (144.97 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 365/7507 본문 확보완료 (145.32 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 366/7507 본문 확보완료 (145.84 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 367/7507 본문 확보완료 (146.25 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 368/7507 본문 확보완료 (146.58 sec)
>>>> Ste

>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 471/7507 본문 확보완료 (188.79 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 472/7507 본문 확보완료 (189.1 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 473/7507 본문 확보완료 (189.64 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 474/7507 본문 확보완료 (189.98 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 475/7507 본문 확보완료 (190.31 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 476/7507 본문 확보완료 (190.71 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 477/7507 본문 확보완료 (191.02 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 478/7507 본문 확보완료 (191.35 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 479/7507 본문 확보완료 (191.71 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 480/7507 본문 확보완료 (192.06 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 481/7507 본문 확보완료 (192.56 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 482/7507 본문 확보완료 (192.94 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 483/7507 본문 확보완료 (193.36 sec)
>>>> Step 3. 정치 영역 국회/정당 주제  20210812 일자 484/7507 본문 확보완료 (193.78 sec)
>>>> St

...

>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7160/7507 본문 확보완료 (2775.76 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7161/7507 본문 확보완료 (2776.26 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7162/7507 본문 확보완료 (2776.58 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7163/7507 본문 확보완료 (2776.94 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7164/7507 본문 확보완료 (2777.26 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7165/7507 본문 확보완료 (2777.53 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7166/7507 본문 확보완료 (2777.81 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7167/7507 본문 확보완료 (2778.15 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7168/7507 본문 확보완료 (2778.42 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7169/7507 본문 확보완료 (2779.08 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7170/7507 본문 확보완료 (2779.32 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7171/7507 본문 확보완료 (2779.6 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7172/7507 본문 확보완료 (2779.91 sec)
>>>> Step 3. 사회 영역 지역 주제  20210813 일자 7173/7507 본문 확보완료 (2780.15 sec)
>>>> Step 3. 사회 영역 지역

>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7276/7507 본문 확보완료 (2820.37 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7277/7507 본문 확보완료 (2821.05 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7278/7507 본문 확보완료 (2821.89 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7279/7507 본문 확보완료 (2822.22 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7280/7507 본문 확보완료 (2822.54 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7281/7507 본문 확보완료 (2822.86 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7282/7507 본문 확보완료 (2823.23 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7283/7507 본문 확보완료 (2823.5 sec)
>>>> Step 3. 생활/문화 영역 도로/교통 주제  20210813 일자 7284/7507 본문 확보완료 (2823.77 sec)
>>>> Step 3. 생활/문화 영역 여행/레저 주제  20210813 일자 7285/7507 본문 확보완료 (2824.21 sec)
>>>> Step 3. 생활/문화 영역 여행/레저 주제  20210813 일자 7286/7507 본문 확보완료 (2824.47 sec)
>>>> Step 3. 생활/문화 영역 여행/레저 주제  20210813 일자 7287/7507 본문 확보완료 (2824.89 sec)
>>>> Step 3. 생활/문화 영역 여행/레저 주제  20210813 일자 7288/7507 본문 확보완료 (2825.22 sec)
>>>> Step 3. 

>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7387/7507 본문 확보완료 (2861.11 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7388/7507 본문 확보완료 (2861.42 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7389/7507 본문 확보완료 (2861.73 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7390/7507 본문 확보완료 (2862.06 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7391/7507 본문 확보완료 (2862.37 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7392/7507 본문 확보완료 (2862.87 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7393/7507 본문 확보완료 (2863.39 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7394/7507 본문 확보완료 (2863.71 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7395/7507 본문 확보완료 (2864.01 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7396/7507 본문 확보완료 (2864.3 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7397/7507 본문 확보완료 (2864.7 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7398/7507 본문 확보완료 (2864.94 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일자 7399/7507 본문 확보완료 (2865.22 sec)
>>>> Step 3. 세계 영역 미국/중남미 주제  20210813 일

>>>> Step 3. IT/과학 영역 컴퓨터 주제  20210813 일자 7497/7507 본문 확보완료 (2898.39 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7498/7507 본문 확보완료 (2898.98 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7499/7507 본문 확보완료 (2899.24 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7500/7507 본문 확보완료 (2899.55 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7501/7507 본문 확보완료 (2899.79 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7502/7507 본문 확보완료 (2900.1 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7503/7507 본문 확보완료 (2900.34 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7504/7507 본문 확보완료 (2900.73 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7505/7507 본문 확보완료 (2900.86 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7506/7507 본문 확보완료 (2900.99 sec)
>>>> Step 3. IT/과학 영역 게임/리뷰 주제  20210813 일자 7507/7507 본문 확보완료 (2901.11 sec)


,DOMAIN,SUB_DOMAIN,DATE,URL,PROC_DVCD,TITLE,ARTICLE,PRESS,W_TIME,WC_VECTOR,NOUN_VECTOR
6698,IT/과학,게임/리뷰,20210812,https://news.naver.com/main/read.naver?mode=LS...,1,,,,,,
7506,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,1,,,,,,
0,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"보험왕 이재명, 춤추는 정세균… 정책 ‘쇼호스트’ 변신한 與후보들",\n\n더불어민주당 대선 경선 후보들이 12일 경기 파주시 한 스튜디오에서 열린 정...,한국일보,202108122330,,
1,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"이준석·김기현, 상주서 회동…당 내홍 수습안 논의(종합)",\n\n\t\n\t(서울=연합뉴스) 이유미 한지훈 기자 = 국민의힘 이준석 대표와 ...,연합뉴스,202108122323,,
2,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"""재명 케어"" ""맞춤형 주거 패키지""…與 주자들 정책 판매","\n\n쇼호스트로 변신한 주자들, 주요 정책 이색 세일즈이재명 ""성남·경기 이어 기...",뉴시스,202108122248,,
...,...,...,...,...,...,...,...,...,...,...,...
7500,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"크래프톤, 2분기 영업이익 1742억원…전년比 0.3%↓","\n\n크래프톤이 올해 2분기 연결기준 매출 4593억원, 영업이익 1742억원, ...",디지털타임스,202108122242,,
7501,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"넷마블, 카카오게임즈 보유주식 3년만에 매도…2000억 차익","\n\n[디지털데일리 왕진화기자] 넷마블(대표 권영식, 이승원)이 카카오게임즈 주식...",디지털데일리,202108122029,,
7502,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"엔씨소프트, 게임사 최초 `ESG 보고서`",\n\n'엔씨소프트 ESG 플레이북 2020' 표지. 엔씨소프트 제공 엔씨소프...,디지털타임스,202108121945,,
7503,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"넷마블, 카겜 지분 전량 매도…3년만에 2천억 차익 남겼다",\n\n\t\n\t[주식 전량 321만8320주 매도]방준혁 넷마블 이사회 의장넷마...,머니투데이,202108121902,,


# ARTICLE pickle save/load

In [9]:
# save
with open('raw_article'+mm+'.pickle', 'wb') as f:
    pickle.dump(article_result, f, pickle.HIGHEST_PROTOCOL)

In [10]:
#test 14부터 시작, 10일부터
#mm = '_0726'
mm

'_0813'

In [11]:
# load
with open('raw_article'+mm+'.pickle', "rb") as f:
    df = pickle.load(f, encoding = 'bytes')
df_article = df.set_index(['DOMAIN']).reset_index()
df_article

,DOMAIN,SUB_DOMAIN,DATE,URL,PROC_DVCD,TITLE,ARTICLE,PRESS,W_TIME,WC_VECTOR,NOUN_VECTOR
0,IT/과학,게임/리뷰,20210812,https://news.naver.com/main/read.naver?mode=LS...,1,,,,,,
1,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,1,,,,,,
2,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"보험왕 이재명, 춤추는 정세균… 정책 ‘쇼호스트’ 변신한 與후보들",\n\n더불어민주당 대선 경선 후보들이 12일 경기 파주시 한 스튜디오에서 열린 정...,한국일보,202108122330,,
3,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"이준석·김기현, 상주서 회동…당 내홍 수습안 논의(종합)",\n\n\t\n\t(서울=연합뉴스) 이유미 한지훈 기자 = 국민의힘 이준석 대표와 ...,연합뉴스,202108122323,,
4,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"""재명 케어"" ""맞춤형 주거 패키지""…與 주자들 정책 판매","\n\n쇼호스트로 변신한 주자들, 주요 정책 이색 세일즈이재명 ""성남·경기 이어 기...",뉴시스,202108122248,,
...,...,...,...,...,...,...,...,...,...,...,...
6316,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"크래프톤, 2분기 영업이익 1742억원…전년比 0.3%↓","\n\n크래프톤이 올해 2분기 연결기준 매출 4593억원, 영업이익 1742억원, ...",디지털타임스,202108122242,,
6317,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"넷마블, 카카오게임즈 보유주식 3년만에 매도…2000억 차익","\n\n[디지털데일리 왕진화기자] 넷마블(대표 권영식, 이승원)이 카카오게임즈 주식...",디지털데일리,202108122029,,
6318,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"엔씨소프트, 게임사 최초 `ESG 보고서`",\n\n'엔씨소프트 ESG 플레이북 2020' 표지. 엔씨소프트 제공 엔씨소프...,디지털타임스,202108121945,,
6319,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"넷마블, 카겜 지분 전량 매도…3년만에 2천억 차익 남겼다",\n\n\t\n\t[주식 전량 321만8320주 매도]방준혁 넷마블 이사회 의장넷마...,머니투데이,202108121902,,


# 불용어 로드

In [12]:
with open('stop_word.pickle', "rb") as f:
    dfs = pickle.load(f, encoding='bytes')
stop_word = list(dfs[0])

# 데이터프레임 조정

In [13]:
#분석대상 제외항목 누락, 어제자 오전 5시 ~ 금일 오전 5시 기사만 출력
# 3가지 조건 적용
ydt = int(yd+'0500')
tdt = int(td+'0500')


df_article['W_TIME'] = pd.to_numeric(df['W_TIME'], errors='coerce').apply(np.floor).astype('Int64')
condition1 = df_article['PROC_DVCD'] == '2'
condition2 = df_article['W_TIME'] >= ydt
condition3 = df_article['W_TIME'] <= tdt
df_article_2 = df_article[condition1 & condition2 & condition3]
df_article_2 = pd.DataFrame(df_article_2)


,DOMAIN,SUB_DOMAIN,DATE,URL,PROC_DVCD,TITLE,ARTICLE,PRESS,W_TIME,WC_VECTOR,NOUN_VECTOR
2,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"보험왕 이재명, 춤추는 정세균… 정책 ‘쇼호스트’ 변신한 與후보들",\n\n더불어민주당 대선 경선 후보들이 12일 경기 파주시 한 스튜디오에서 열린 정...,한국일보,202108122330,,
3,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"이준석·김기현, 상주서 회동…당 내홍 수습안 논의(종합)",\n\n\t\n\t(서울=연합뉴스) 이유미 한지훈 기자 = 국민의힘 이준석 대표와 ...,연합뉴스,202108122323,,
4,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"""재명 케어"" ""맞춤형 주거 패키지""…與 주자들 정책 판매","\n\n쇼호스트로 변신한 주자들, 주요 정책 이색 세일즈이재명 ""성남·경기 이어 기...",뉴시스,202108122248,,
5,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,최재형의 재반박 “국가가 국민의 모든 삶 책임진다는 건 전체주의”,\n\n\t\n\t\t\t\t\t\t\t\t“국민의 삶을 국민이 책임져야지 왜 정부...,조선일보,202108122239,,
6,정치,국회/정당,20210812,https://news.naver.com/main/read.naver?mode=LS...,2,"정의당 “與의 언론법은 트럼프식 증오법, 꼭 막겠다”","\n\n황희 문화체육관광부 장관, 오영우 제1차관이 12일 서울 여의도 국회에서 열...",조선일보,202108122238,,
...,...,...,...,...,...,...,...,...,...,...,...
6315,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"'제2의 나라', 클래스 밸런스 개선 업데이트 실시",\n\n[OSEN=고용준 기자] 넷마블의 인기 게임 '제2의 나라'가 유저들의 의견...,OSEN,202108122333,,
6316,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"크래프톤, 2분기 영업이익 1742억원…전년比 0.3%↓","\n\n크래프톤이 올해 2분기 연결기준 매출 4593억원, 영업이익 1742억원, ...",디지털타임스,202108122242,,
6317,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"넷마블, 카카오게임즈 보유주식 3년만에 매도…2000억 차익","\n\n[디지털데일리 왕진화기자] 넷마블(대표 권영식, 이승원)이 카카오게임즈 주식...",디지털데일리,202108122029,,
6318,IT/과학,게임/리뷰,20210813,https://news.naver.com/main/read.naver?mode=LS...,2,"엔씨소프트, 게임사 최초 `ESG 보고서`",\n\n'엔씨소프트 ESG 플레이북 2020' 표지. 엔씨소프트 제공 엔씨소프...,디지털타임스,202108121945,,


# ARTICLE

In [14]:
okt = Okt()
vector = CountVectorizer()


df_article_3 = {'DOMAIN' : [], 'SUB_DOMAIN':[], 'PRESS':[], 'TITLE': [], 'URL':[], 'ARTICLE': [], 'W_TIME': [], 'COUNT_WORD':[], 'WORD_LIST': [], }
for idx,row in df_article_2.iterrows():        
    domain = row['DOMAIN']
    sub_domain = row['SUB_DOMAIN']
    press = row['PRESS']
    title = row['TITLE']
    url = row['URL']
    article = row['ARTICLE']
    w_time = row['W_TIME']
    
    try:
        noun = okt.nouns(df_article_2['ARTICLE'][idx])
        noun_adj = [x for x in noun if x not in stop_word]
        noun_adj = ", ".join(noun_adj)
        b = vector.fit_transform([noun_adj]).toarray().tolist()
        b = sum(b, []) #리스트 축소
        a = vector.get_feature_names()
        wc = {name:value for name, value in zip(a, b)}
        wc = dict(wc)
        wc = dict(sorted(wc.items(), key=lambda x : x[1],reverse=True))
        
    except:
        wc='' #영어 기사 본문 출력 안됨, 널값처리 필요

    df_article_3['DOMAIN'].append(domain)
    df_article_3['SUB_DOMAIN'].append(sub_domain)
    df_article_3['PRESS'].append(press)
    df_article_3['TITLE'].append(title)
    df_article_3['URL'].append(url)
    df_article_3['ARTICLE'].append(article)
    df_article_3['W_TIME'].append(w_time)
    df_article_3['COUNT_WORD'].append(wc)
    df_article_3['WORD_LIST'].append(noun_adj)

df_article_3 = pd.DataFrame(df_article_3)
df_article_3 = df_article_3.drop_duplicates(["DOMAIN","TITLE"], keep="last")



In [15]:
with open('adj_article'+mm+'.pickle', 'wb') as f:
    pickle.dump(df_article_3, f, pickle.HIGHEST_PROTOCOL)

# adj-article load

In [13]:
# load
with open('adj_article'+mm+'.pickle', "rb") as f:
    df = pickle.load(f, encoding = 'bytes')
df_article_3 = df.set_index(['DOMAIN']).reset_index()
df_article_3

,DOMAIN,SUB_DOMAIN,PRESS,TITLE,URL,ARTICLE,W_TIME,COUNT_WORD,WORD_LIST
0,정치,국회/정당,한국일보,"보험왕 이재명, 춤추는 정세균… 정책 ‘쇼호스트’ 변신한 與후보들",https://news.naver.com/main/read.naver?mode=LS...,\n\n더불어민주당 대선 경선 후보들이 12일 경기 파주시 한 스튜디오에서 열린 정...,202108122330,"{'대선': 6, '마켓': 6, '커머스': 6, '파주시': 5, '더민': 4...","대선, 파주시, 커머스, 더, 민, 마켓, 완판, 이재명, 김두관, 정세균, 이낙연..."
1,정치,국회/정당,연합뉴스,"이준석·김기현, 상주서 회동…당 내홍 수습안 논의(종합)",https://news.naver.com/main/read.naver?mode=LS...,\n\n\t\n\t(서울=연합뉴스) 이유미 한지훈 기자 = 국민의힘 이준석 대표와 ...,202108122323,"{'원내대표': 5, '준위': 5, '당내': 3, '김기현': 2, '월권': ...","유미, 한지훈, 힘, 이준석, 김기현, 원내대표, 준위, 월권, 당내, 회동했다, ..."
2,정치,국회/정당,뉴시스,"""재명 케어"" ""맞춤형 주거 패키지""…與 주자들 정책 판매",https://news.naver.com/main/read.naver?mode=LS...,"\n\n쇼호스트로 변신한 주자들, 주요 정책 이색 세일즈이재명 ""성남·경기 이어 기...",202108122248,"{'이재명': 5, '기본소득': 4, '대선': 4, '마켓': 4, '이낙연':...","쇼호, 스트로, 변신, 이색, 이재명, 성남, 기본소득, 완판, 신화, 들것, 이낙..."
3,정치,국회/정당,조선일보,최재형의 재반박 “국가가 국민의 모든 삶 책임진다는 건 전체주의”,https://news.naver.com/main/read.naver?mode=LS...,\n\n\t\n\t\t\t\t\t\t\t\t“국민의 삶을 국민이 책임져야지 왜 정부...,202108122239,"{'대선': 5, '간섭': 4, '개입': 3, '전체주의': 3, '정치권': ...","삶, 왜, 힘, 대선, 최재형, 전, 감사원장, 정치권, 연일, 방이, 여당, 힘,..."
4,정치,국회/정당,조선일보,"정의당 “與의 언론법은 트럼프식 증오법, 꼭 막겠다”",https://news.naver.com/main/read.naver?mode=LS...,"\n\n황희 문화체육관광부 장관, 오영우 제1차관이 12일 서울 여의도 국회에서 열...",202108122238,"{'손해배상': 5, '야당': 5, '징벌': 5, '민주주의': 3, '이쯤':...","황희, 영, 제, 관광, 개, 를, 관광, 징벌, 법, 중재, 및, 구제, 등, 관..."
...,...,...,...,...,...,...,...,...,...
5022,IT/과학,게임/리뷰,OSEN,"'제2의 나라', 클래스 밸런스 개선 업데이트 실시",https://news.naver.com/main/read.naver?mode=LS...,\n\n[OSEN=고용준 기자] 넷마블의 인기 게임 '제2의 나라'가 유저들의 의견...,202108122333,"{'넷마블': 4, '스킬': 4, '밸런스': 3, '일일': 2, '고용': 1...","고용, 준, 넷마블, 제, 의, 밸런스, 넷마블, 제, 의, 제, 의, 사, 넷마블..."
5023,IT/과학,게임/리뷰,디지털타임스,"크래프톤, 2분기 영업이익 1742억원…전년比 0.3%↓",https://news.naver.com/main/read.naver?mode=LS...,"\n\n크래프톤이 올해 2분기 연결기준 매출 4593억원, 영업이익 1742억원, ...",202108122242,"{'라운드': 5, '배틀': 5, '이익': 3, '글로벌': 2, '크래프톤':...","크래프톤, 이익, 순이익, 년, 이익, 중, 글로벌, 를, 액, 펍, 배틀, 라운드..."
5024,IT/과학,게임/리뷰,디지털데일리,"넷마블, 카카오게임즈 보유주식 3년만에 매도…2000억 차익",https://news.naver.com/main/read.naver?mode=LS...,"\n\n[디지털데일리 왕진화기자] 넷마블(대표 권영식, 이승원)이 카카오게임즈 주식...",202108122029,"{'넷마블': 6, '대금': 2, '매도': 2, '스핀': 2, '엑스': 2,...","데, 왕진, 넷마블, 권영식, 이승원, 이, 전량, 주, 매도, 의, 종가, 시, ..."
5025,IT/과학,게임/리뷰,디지털타임스,"엔씨소프트, 게임사 최초 `ESG 보고서`",https://news.naver.com/main/read.naver?mode=LS...,\n\n'엔씨소프트 ESG 플레이북 2020' 표지. 엔씨소프트 제공 엔씨소프...,202108121945,"{'글로벌': 2, '발간': 2, '부활동': 2, '전과': 2, '지배구조':...","북, 사, 지배구조, 비, 전과, 북, 발간, 페이지, 총, 페이지, 달, 기업가,..."


In [14]:
#결측값제거

df_article_4 = df_article_3.dropna(axis=0)
df_article_4

,DOMAIN,SUB_DOMAIN,PRESS,TITLE,URL,ARTICLE,W_TIME,COUNT_WORD,WORD_LIST
0,정치,국회/정당,한국일보,"보험왕 이재명, 춤추는 정세균… 정책 ‘쇼호스트’ 변신한 與후보들",https://news.naver.com/main/read.naver?mode=LS...,\n\n더불어민주당 대선 경선 후보들이 12일 경기 파주시 한 스튜디오에서 열린 정...,202108122330,"{'대선': 6, '마켓': 6, '커머스': 6, '파주시': 5, '더민': 4...","대선, 파주시, 커머스, 더, 민, 마켓, 완판, 이재명, 김두관, 정세균, 이낙연..."
1,정치,국회/정당,연합뉴스,"이준석·김기현, 상주서 회동…당 내홍 수습안 논의(종합)",https://news.naver.com/main/read.naver?mode=LS...,\n\n\t\n\t(서울=연합뉴스) 이유미 한지훈 기자 = 국민의힘 이준석 대표와 ...,202108122323,"{'원내대표': 5, '준위': 5, '당내': 3, '김기현': 2, '월권': ...","유미, 한지훈, 힘, 이준석, 김기현, 원내대표, 준위, 월권, 당내, 회동했다, ..."
2,정치,국회/정당,뉴시스,"""재명 케어"" ""맞춤형 주거 패키지""…與 주자들 정책 판매",https://news.naver.com/main/read.naver?mode=LS...,"\n\n쇼호스트로 변신한 주자들, 주요 정책 이색 세일즈이재명 ""성남·경기 이어 기...",202108122248,"{'이재명': 5, '기본소득': 4, '대선': 4, '마켓': 4, '이낙연':...","쇼호, 스트로, 변신, 이색, 이재명, 성남, 기본소득, 완판, 신화, 들것, 이낙..."
3,정치,국회/정당,조선일보,최재형의 재반박 “국가가 국민의 모든 삶 책임진다는 건 전체주의”,https://news.naver.com/main/read.naver?mode=LS...,\n\n\t\n\t\t\t\t\t\t\t\t“국민의 삶을 국민이 책임져야지 왜 정부...,202108122239,"{'대선': 5, '간섭': 4, '개입': 3, '전체주의': 3, '정치권': ...","삶, 왜, 힘, 대선, 최재형, 전, 감사원장, 정치권, 연일, 방이, 여당, 힘,..."
4,정치,국회/정당,조선일보,"정의당 “與의 언론법은 트럼프식 증오법, 꼭 막겠다”",https://news.naver.com/main/read.naver?mode=LS...,"\n\n황희 문화체육관광부 장관, 오영우 제1차관이 12일 서울 여의도 국회에서 열...",202108122238,"{'손해배상': 5, '야당': 5, '징벌': 5, '민주주의': 3, '이쯤':...","황희, 영, 제, 관광, 개, 를, 관광, 징벌, 법, 중재, 및, 구제, 등, 관..."
...,...,...,...,...,...,...,...,...,...
5022,IT/과학,게임/리뷰,OSEN,"'제2의 나라', 클래스 밸런스 개선 업데이트 실시",https://news.naver.com/main/read.naver?mode=LS...,\n\n[OSEN=고용준 기자] 넷마블의 인기 게임 '제2의 나라'가 유저들의 의견...,202108122333,"{'넷마블': 4, '스킬': 4, '밸런스': 3, '일일': 2, '고용': 1...","고용, 준, 넷마블, 제, 의, 밸런스, 넷마블, 제, 의, 제, 의, 사, 넷마블..."
5023,IT/과학,게임/리뷰,디지털타임스,"크래프톤, 2분기 영업이익 1742억원…전년比 0.3%↓",https://news.naver.com/main/read.naver?mode=LS...,"\n\n크래프톤이 올해 2분기 연결기준 매출 4593억원, 영업이익 1742억원, ...",202108122242,"{'라운드': 5, '배틀': 5, '이익': 3, '글로벌': 2, '크래프톤':...","크래프톤, 이익, 순이익, 년, 이익, 중, 글로벌, 를, 액, 펍, 배틀, 라운드..."
5024,IT/과학,게임/리뷰,디지털데일리,"넷마블, 카카오게임즈 보유주식 3년만에 매도…2000억 차익",https://news.naver.com/main/read.naver?mode=LS...,"\n\n[디지털데일리 왕진화기자] 넷마블(대표 권영식, 이승원)이 카카오게임즈 주식...",202108122029,"{'넷마블': 6, '대금': 2, '매도': 2, '스핀': 2, '엑스': 2,...","데, 왕진, 넷마블, 권영식, 이승원, 이, 전량, 주, 매도, 의, 종가, 시, ..."
5025,IT/과학,게임/리뷰,디지털타임스,"엔씨소프트, 게임사 최초 `ESG 보고서`",https://news.naver.com/main/read.naver?mode=LS...,\n\n'엔씨소프트 ESG 플레이북 2020' 표지. 엔씨소프트 제공 엔씨소프...,202108121945,"{'글로벌': 2, '발간': 2, '부활동': 2, '전과': 2, '지배구조':...","북, 사, 지배구조, 비, 전과, 북, 발간, 페이지, 총, 페이지, 달, 기업가,..."


In [16]:
df_article_5 = {'DOMAIN' : [], 'SUB_DOMAIN':[], 'PRESS':[], 'TITLE': [], 'URL' : [], 'W_TIME' : [], 'MM' : [], 'COUNT_WORD':[], 'CUM_COUNT_WORD' :[], 'CUM_COUNT_WORD_TOP20' :[], 'WORD_LIST' : []}         
list_test = df_article_4['DOMAIN'].unique().tolist()


for i in range(len(list_test)):
    info_list_test = df_article_4[df_article_4['DOMAIN'] ==  list_test[i]]
    info_list_test.reset_index(drop=True, inplace=True)
    cum_count_word = Counter(dict(info_list_test['COUNT_WORD'][0]))

    for idx,row in info_list_test.iterrows():
        domain = row['DOMAIN']
        sub_domain = row['SUB_DOMAIN']
        press = row['PRESS']
        title = row['TITLE']
        url = row['URL']
        w_time = row['W_TIME']
        count_word = row['COUNT_WORD']
        word_list = row['WORD_LIST']
        
        if idx == 0:
            cwc2 = count_word
            cwc3 = count_word
        else :
            cum_count_word = cum_count_word + Counter(dict(info_list_test['COUNT_WORD'][idx]))
            cwc1 = dict(cum_count_word)
            cwc2 = dict(sorted(cwc1.items(), key=lambda x : x[1],reverse=True))
            cwc3 = dict(sorted(cwc1.items(), key=lambda x : x[1],reverse=True)[:20])
        df_article_5['DOMAIN'].append(domain)
        df_article_5['SUB_DOMAIN'].append(sub_domain)
        df_article_5['PRESS'].append(press)
        df_article_5['TITLE'].append(title)
        df_article_5['URL'].append(url)
        df_article_5['W_TIME'].append(w_time)
        df_article_5['MM'].append(mm)
        df_article_5['COUNT_WORD'].append(count_word)
        df_article_5['CUM_COUNT_WORD'].append(cwc2)
        df_article_5['CUM_COUNT_WORD_TOP20'].append(cwc3)
        df_article_5['WORD_LIST'].append(word_list)
    print(">>>>  {} 영역  완료했습니다. ({} sec)".format(domain, sub_domain))
    i+=1
    
df_article_5 = pd.DataFrame(df_article_5)

>>>>  정치 영역  완료했습니다. (국방/외교 sec)
>>>>  경제 영역  완료했습니다. (글로벌 경제 sec)
>>>>  사회 영역  완료했습니다. (인물 sec)
>>>>  생활/문화 영역  완료했습니다. (날씨 sec)
>>>>  세계 영역  완료했습니다. (중동/아프리카 sec)
>>>>  IT/과학 영역  완료했습니다. (게임/리뷰 sec)


# 카운팅 DataFrame
######          -영역별 누적 키워드 카운트

In [17]:
df_domain_count = df_article_5.drop_duplicates(['DOMAIN'], keep='last')[['DOMAIN','CUM_COUNT_WORD','CUM_COUNT_WORD_TOP20']]

In [18]:
df_domain_count.reset_index(drop=True, inplace=True)
df_domain_count

,DOMAIN,CUM_COUNT_WORD,CUM_COUNT_WORD_TOP20
0,정치,"{'대선': 586, '캠프': 563, '대통령': 474, '윤석열': 322,...","{'대선': 586, '캠프': 563, '대통령': 474, '윤석열': 322,..."
1,경제,"{'삼성': 844, '코로나': 756, '이익': 638, '미국': 599, ...","{'삼성': 844, '코로나': 756, '이익': 638, '미국': 599, ..."
2,사회,"{'코로나': 1876, '백신': 558, '광주': 433, '울산': 394,...","{'코로나': 1876, '백신': 558, '광주': 433, '울산': 394,..."
3,생활/문화,"{'자동차': 161, '볼트': 154, '전기차': 122, '코로나': 110...","{'자동차': 161, '볼트': 154, '전기차': 122, '코로나': 110..."
4,세계,"{'미국': 380, '백신': 340, '코로나': 191, '대통령': 163,...","{'미국': 380, '백신': 340, '코로나': 191, '대통령': 163,..."
5,IT/과학,"{'갤럭시': 675, '글로벌': 390, '삼성': 341, '이익': 246,...","{'갤럭시': 675, '글로벌': 390, '삼성': 341, '이익': 246,..."


In [19]:
# save
with open('domain_word'+mm+'.pickle', 'wb') as f:
    pickle.dump(df_domain_count, f, pickle.HIGHEST_PROTOCOL)

In [20]:
# load
with open('domain_word'+mm+'.pickle', 'rb') as f:
    df = pickle.load(f)
df_domain_count= df.set_index(['DOMAIN']).reset_index()
df_domain_count


,DOMAIN,CUM_COUNT_WORD,CUM_COUNT_WORD_TOP20
0,정치,"{'대선': 586, '캠프': 563, '대통령': 474, '윤석열': 322,...","{'대선': 586, '캠프': 563, '대통령': 474, '윤석열': 322,..."
1,경제,"{'삼성': 844, '코로나': 756, '이익': 638, '미국': 599, ...","{'삼성': 844, '코로나': 756, '이익': 638, '미국': 599, ..."
2,사회,"{'코로나': 1876, '백신': 558, '광주': 433, '울산': 394,...","{'코로나': 1876, '백신': 558, '광주': 433, '울산': 394,..."
3,생활/문화,"{'자동차': 161, '볼트': 154, '전기차': 122, '코로나': 110...","{'자동차': 161, '볼트': 154, '전기차': 122, '코로나': 110..."
4,세계,"{'미국': 380, '백신': 340, '코로나': 191, '대통령': 163,...","{'미국': 380, '백신': 340, '코로나': 191, '대통령': 163,..."
5,IT/과학,"{'갤럭시': 675, '글로벌': 390, '삼성': 341, '이익': 246,...","{'갤럭시': 675, '글로벌': 390, '삼성': 341, '이익': 246,..."


# tf-idf 적용위한 DataFrame

In [21]:
df_tfidf = df_article_5[['DOMAIN','MM','TITLE','URL','WORD_LIST', 'CUM_COUNT_WORD_TOP20']]
with open('tfidf_data'+mm+'.pickle', 'wb') as f:
    pickle.dump(df_tfidf, f, pickle.HIGHEST_PROTOCOL)

In [22]:
with open('tfidf_data'+mm+'.pickle', 'rb') as f:
    df_tfidf = pickle.load(f)
df_tfidf

,DOMAIN,MM,TITLE,URL,WORD_LIST,CUM_COUNT_WORD_TOP20
0,정치,_0813,"보험왕 이재명, 춤추는 정세균… 정책 ‘쇼호스트’ 변신한 與후보들",https://news.naver.com/main/read.naver?mode=LS...,"대선, 파주시, 커머스, 더, 민, 마켓, 완판, 이재명, 김두관, 정세균, 이낙연...","{'대선': 6, '마켓': 6, '커머스': 6, '파주시': 5, '더민': 4..."
1,정치,_0813,"이준석·김기현, 상주서 회동…당 내홍 수습안 논의(종합)",https://news.naver.com/main/read.naver?mode=LS...,"유미, 한지훈, 힘, 이준석, 김기현, 원내대표, 준위, 월권, 당내, 회동했다, ...","{'대선': 7, '마켓': 6, '커머스': 6, '파주시': 5, '원내대표':..."
2,정치,_0813,"""재명 케어"" ""맞춤형 주거 패키지""…與 주자들 정책 판매",https://news.naver.com/main/read.naver?mode=LS...,"쇼호, 스트로, 변신, 이색, 이재명, 성남, 기본소득, 완판, 신화, 들것, 이낙...","{'대선': 11, '마켓': 10, '커머스': 9, '파주시': 8, '이재명'..."
3,정치,_0813,최재형의 재반박 “국가가 국민의 모든 삶 책임진다는 건 전체주의”,https://news.naver.com/main/read.naver?mode=LS...,"삶, 왜, 힘, 대선, 최재형, 전, 감사원장, 정치권, 연일, 방이, 여당, 힘,...","{'대선': 16, '마켓': 10, '커머스': 9, '파주시': 8, '이재명'..."
4,정치,_0813,"정의당 “與의 언론법은 트럼프식 증오법, 꼭 막겠다”",https://news.naver.com/main/read.naver?mode=LS...,"황희, 영, 제, 관광, 개, 를, 관광, 징벌, 법, 중재, 및, 구제, 등, 관...","{'대선': 18, '마켓': 10, '커머스': 9, '파주시': 8, '이재명'..."
...,...,...,...,...,...,...
5022,IT/과학,_0813,"'제2의 나라', 클래스 밸런스 개선 업데이트 실시",https://news.naver.com/main/read.naver?mode=LS...,"고용, 준, 넷마블, 제, 의, 밸런스, 넷마블, 제, 의, 제, 의, 사, 넷마블...","{'갤럭시': 675, '글로벌': 386, '삼성': 341, '이익': 243,..."
5023,IT/과학,_0813,"크래프톤, 2분기 영업이익 1742억원…전년比 0.3%↓",https://news.naver.com/main/read.naver?mode=LS...,"크래프톤, 이익, 순이익, 년, 이익, 중, 글로벌, 를, 액, 펍, 배틀, 라운드...","{'갤럭시': 675, '글로벌': 388, '삼성': 341, '이익': 246,..."
5024,IT/과학,_0813,"넷마블, 카카오게임즈 보유주식 3년만에 매도…2000억 차익",https://news.naver.com/main/read.naver?mode=LS...,"데, 왕진, 넷마블, 권영식, 이승원, 이, 전량, 주, 매도, 의, 종가, 시, ...","{'갤럭시': 675, '글로벌': 388, '삼성': 341, '이익': 246,..."
5025,IT/과학,_0813,"엔씨소프트, 게임사 최초 `ESG 보고서`",https://news.naver.com/main/read.naver?mode=LS...,"북, 사, 지배구조, 비, 전과, 북, 발간, 페이지, 총, 페이지, 달, 기업가,...","{'갤럭시': 675, '글로벌': 390, '삼성': 341, '이익': 246,..."


# 하루전

In [23]:
with open('tfidf_data'+mm1+'.pickle', 'rb') as f:
    df_tfidf_1 = pickle.load(f)
df_tfidf_1

,DOMAIN,MM,TITLE,URL,WORD_LIST,CUM_COUNT_WORD_TOP20
0,정치,_0812,"끝내 폭발한 이재명 ""이게 네거티브""…'명낙대전 일촉즉발'",https://news.naver.com/main/read.naver?mode=LS...,"이재명, 이낙연, 내내, 기본소득, 사드, 등, 충돌, 이낙연, 이재명, 프레임, ...","{'이재명': 11, '사드': 9, '이낙연': 8, '기본소득': 5, '대선'..."
1,정치,_0812,"유승민 ""이준석, 말 줄여야""…원희룡 ""경준위, 너무 앞지르는 독단""",https://news.naver.com/main/read.naver?mode=LS...,"유승민, 박근혜, 수감, 원희룡, 대선, 말, 해, 유승민, 전, 주연, 김유, 승...","{'이재명': 11, '사드': 9, '대선': 9, '이낙연': 8, '준위': ..."
2,정치,_0812,"송두환 인권위원장 후보자, 재산 32억9천만원 신고",https://news.naver.com/main/read.naver?mode=LS...,"문, 대통령, 국가인권위, 회, 송두환, 변호사, 내정, 상민, 송두환, 국가인권위...","{'이재명': 11, '사드': 9, '대선': 9, '이낙연': 8, '준위': ..."
3,정치,_0812,"송두환 인권위원장 후보자, 재산 32억원 신고…대치동 아파트 등 보유",https://news.naver.com/main/read.naver?mode=LS...,"남양주, 등, 인권, 신념, 인권위, 적임, 문재인, 대통령, 국가인권위, 회, 후...","{'후보자': 15, '이재명': 11, '사드': 9, '대선': 9, '대통령'..."
4,정치,_0812,"최재형 ""이낙연, 조국기 부대에 아부…청년 상실감 안 보이나""",https://news.naver.com/main/read.naver?mode=LS...,"일가, 위선, 분열, 상처, 전신, 힘, 대권, 최재형, 전, 감사원장, 힘, 초선...","{'후보자': 15, '대통령': 12, '이재명': 11, '이낙연': 10, '..."
...,...,...,...,...,...,...
5169,IT/과학,_0812,불법 스팸문자 전송 수법 조심하세요,https://news.naver.com/main/read.naver?mode=LS...,"를, 알바생, 스팸, 인터넷, 과, 중고생, 유혹, 스팸, 수법, 스팸, 세력, 알...","{'갤럭시': 1065, '삼성': 462, '글로벌': 411, '이익': 382..."
5170,IT/과학,_0812,SW교육 비중 '0.4%'…지금 안 늘리면 7년 더 뒤처진다,https://news.naver.com/main/read.naver?mode=LS...,"수, 공교육, 수, 소프트웨어, 그, 후, 또, 그냥, 거, 컴퓨터, 바, 저절로,...","{'갤럭시': 1065, '삼성': 462, '글로벌': 412, '이익': 382..."
5171,IT/과학,_0812,"트러스트버스, 마스월렛·마스터키 iOS 버전 출시",https://news.naver.com/main/read.naver?mode=LS...,"블록, 체인, 금융, 트러스트, 버스, 암호, 화폐, 지갑, 마스, 월렛, 과, 프...","{'갤럭시': 1065, '삼성': 462, '글로벌': 413, '이익': 382..."
5172,IT/과학,_0812,인터파크·다나와 매각 행렬…e커머스 재편에 저무는 1세대,https://news.naver.com/main/read.naver?mode=LS...,"재편, 지적도, 데, 이안, 커머스, 신세계, 등, 재, 달, 매물, 커머스, 처,...","{'갤럭시': 1065, '삼성': 462, '글로벌': 413, '이익': 383..."


# 이틀전

In [24]:
with open('tfidf_data'+mm2+'.pickle', 'rb') as f:
    df_tfidf_2 = pickle.load(f)
df_tfidf_2

,DOMAIN,MM,TITLE,URL,WORD_LIST,CUM_COUNT_WORD_TOP20
0,정치,_0811,"이재명 ""비난받던 정책들로 인정받아…그냥 포퓰리즘하겠다""",https://news.naver.com/main/read.naver?mode=LS...,"대권, 이재명, 이동형, 김동호, 대권, 이재명, 포퓰리즘, 비난, 것, 것, 앞,...","{'포퓰리즘': 5, '대권': 2, '세금': 2, '이동형': 2, '이재명':..."
1,정치,_0811,"김두관 ""아수라백작"" 저격에…이낙연측 ""선 넘지 마라""",https://news.naver.com/main/read.naver?mode=LS...,"대선, 상암동, 대선, 정세균, 이낙연, 추미애, 김두관, 이재명, 상민, 대권, ...","{'김두관': 6, '이낙연': 6, '포퓰리즘': 5, '대권': 3, '이재명'..."
2,정치,_0811,"김의겸 ""尹사단 검사 2명, 조국에 미안하다 전화""",https://news.naver.com/main/read.naver?mode=LS...,"김, 의겸, 진성, 철, 김, 의겸, 관광, 김동호, 김, 의겸, 자타, 윤석열, ...","{'김두관': 6, '이낙연': 6, '포퓰리즘': 5, '대권': 3, '이재명'..."
3,정치,_0811,정의당도 언론징벌법 반대... 與는 강행 수순,https://news.naver.com/main/read.naver?mode=LS...,"중재, 및, 구제, 등, 관, 정의당, 당론, 민언련, 등, 친, 과잉, 단독, 태...","{'김두관': 6, '이낙연': 6, '포퓰리즘': 5, '정의당': 5, '중재법..."
4,정치,_0811,"文 “문자폭탄은 양념”이라는데... 송영길은 “배설물, 거리둬야”",https://news.naver.com/main/read.naver?mode=LS...,"송영길, 대선, 지지자, 폭탄, 배설물, 말, 아예, 무시, 고, 문재인, 대통령,...","{'대선': 6, '김두관': 6, '이낙연': 6, '포퓰리즘': 5, '정의당'..."
...,...,...,...,...,...,...
4854,IT/과학,_0811,"컴투스, 컴퍼니 빌더 '게임체인저' 지분 40% 확보",https://news.naver.com/main/read.naver?mode=LS...,"컴투스, 체인, 저, 이, 하이, 더, 컴투스, 체인, 저, 해, 분, 를, 체인,...","{'보안': 293, '글로벌': 247, '코로나': 239, '스마트': 219..."
4855,IT/과학,_0811,"넷마블, 카뱅 주식으로 3900억 벌었다",https://news.naver.com/main/read.naver?mode=LS...,"넷마블, 전, 뱅크, 팔, 약, 달, 차익, 것, 넷마블, 뱅크, 장내, 매도, 이...","{'보안': 293, '글로벌': 249, '코로나': 239, '스마트': 219..."
4856,IT/과학,_0811,"넷마블, 카뱅 주식 440억에 사서 4300억에 팔았다",https://news.naver.com/main/read.naver?mode=LS...,"뱅크, 방준혁, 넷마블, 넷마블, 뱅크, 매도, 로만, 약, 차익, 것, 넷마블, ...","{'보안': 293, '글로벌': 249, '코로나': 239, '스마트': 219..."
4857,IT/과학,_0811,"넷마블, 카카오뱅크 지분 1.8% 매각…시세 차익 3900억 벌어",https://news.naver.com/main/read.naver?mode=LS...,"데, 왕진, 넷마블, 권영식, 이승원, 이, 뱅크, 매도, 넷마블, 뱅크, 분, 중...","{'보안': 293, '글로벌': 249, '코로나': 239, '스마트': 219..."


# 사흘전

In [25]:
with open('tfidf_data'+mm3+'.pickle', 'rb') as f:
    df_tfidf_3 = pickle.load(f)
df_tfidf_3

,DOMAIN,MM,TITLE,URL,WORD_LIST,CUM_COUNT_WORD_TOP20
0,정치,_0810,"이재용 가석방 놓고 엇갈린 與 주자들…""불이익 없어야"" vs ""곱빼기 특혜""(종합)",https://news.naver.com/main/read.naver?mode=LS...,"이재명, 캠프, 재벌, 불이익, 선, 정세균, 이재용, 반성, 이바지, 추미애, 시...","{'가석방': 16, '부회장': 8, '이재명': 6, '재벌': 6, '이재용'..."
1,정치,_0810,"안민석 ""조상 친일행각 해명하라""…최재형측 ""허언증 또 도졌냐""",https://news.naver.com/main/read.naver?mode=LS...,"안민석, 최재형, 독립운동가, 후손, 행세, 만주군, 항일, 투사, 둔갑, 안민석,...","{'가석방': 16, '최재형': 10, '부회장': 8, '이재명': 6, '재벌..."
2,정치,_0810,"""조상이 친일?"" 안민석 주장에…崔측 ""허언증 도졌나""",https://news.naver.com/main/read.naver?mode=LS...,"선거, 힘, 최재형, 대선, 헌, 힘, 최재형, 대선, 대하, 빌딩, 선거, 선거,...","{'가석방': 16, '최재형': 13, '조부': 10, '부회장': 8, '증조..."
3,정치,_0810,"""이재명 당선에 음으로 힘보태라"" 이낙연측에 협박 메일…경찰 수사",https://news.naver.com/main/read.naver?mode=LS...,"자칭, 이재명, 지지자, 이낙연, 캠프, 윤영, 찬, 메일, 성범죄, 등, 윤영, ...","{'가석방': 16, '최재형': 13, '이재명': 12, '메일': 11, '조..."
4,정치,_0810,"유승민, 중도·수도권·청년 공략 ‘중수청 캠프’ 발족",https://news.naver.com/main/read.naver?mode=LS...,"환, 김세연, 신원식, 이종훈, 합류, 힘, 유승민, 전, 캠프, 대선, 캠프, 발...","{'캠프': 17, '가석방': 16, '최재형': 13, '이재명': 12, '메..."
...,...,...,...,...,...,...
4771,IT/과학,_0810,"게임빌 드래곤블레이즈, 세컨드임팩트 ‘크룬-새턴’ 업데이트",https://news.naver.com/main/read.naver?mode=LS...,"모험가, 터, 게임빌, 용국, 이, 글로벌, 히트, 작, 드래곤, 블레이즈, 의, ...","{'글로벌': 158, '부회장': 153, '클라우드': 146, '삼성': 13..."
4772,IT/과학,_0810,우중충한 고시촌에 아이디어와 기술이 넘쳐난다고? [스타트업 리포트],https://news.naver.com/main/read.naver?mode=LS...,"관악, 관악구, 호암로, 한국판, 를, 관악, 관악, 신생, 미국, 스탠퍼드, 근처...","{'글로벌': 158, '부회장': 153, '클라우드': 146, '삼성': 13..."
4773,IT/과학,_0810,中규제 엎친데 ‘사내 성폭행’ 덮친 알리바바,https://news.naver.com/main/read.naver?mode=LS...,"사, 방관, 알리바바, 책임자, 해고, 중국, 전자상거래, 알리바바, 창업, 마윈,...","{'글로벌': 158, '부회장': 153, '클라우드': 146, '삼성': 13..."
4774,IT/과학,_0810,나노입자 ‘리간드분자’ 3차원 분포 첫 규명,https://news.naver.com/main/read.naver?mode=LS...,"최, 벽파, 팀, 상반, 포, 용해, 카이스트, 신소재공학, 최, 벽파, 팀, 나노...","{'글로벌': 158, '부회장': 153, '클라우드': 146, '삼성': 13..."


# 나흘전

In [26]:
with open('tfidf_data'+mm4+'.pickle', 'rb') as f:
    df_tfidf_4 = pickle.load(f)
df_tfidf_4

,DOMAIN,MM,TITLE,URL,WORD_LIST,CUM_COUNT_WORD_TOP20
0,정치,_0809,"이재명·이낙연 ""올림픽 선수단, 승패 상관없이 즐기는 모습 신선""",https://news.naver.com/main/read.naver?mode=LS...,"이재명, 이재명, 이낙연, 투혼, 후회, 것, 도쿄, 국립경기장, 도쿄올림픽, 폐막...","{'메달': 5, '이재명': 5, '올림픽': 4, '이낙연': 3, '코로나':..."
1,정치,_0809,"이낙연 ""韓선수단, '낡은' 성적 지상주의 날려버렸다""",https://news.naver.com/main/read.naver?mode=LS...,"인천공항, 고승민, 도쿄올림픽, 육상, 높이뛰기, 위, 신기록, 우상, 혁, 국군체...","{'메달': 5, '이재명': 5, '올림픽': 5, '코로나': 5, '도쿄올림픽..."
2,정치,_0809,이준석 “불문에 붙일 것” 윤석열 “다음엔 참석” 당행사 보이콧 논란 봉합?,https://news.naver.com/main/read.naver?mode=LS...,"힘, 이준석, 입당, 윤석열, 접견, 후, 대선, 완전, 힘, 이준석, 윤석열, 전...","{'메달': 5, '이재명': 5, '올림픽': 5, '코로나': 5, '도쿄올림픽..."
3,정치,_0809,안동 찾은 국민의힘 이준석 대표,https://news.naver.com/main/read.naver?mode=LS...,"박홍식, 힘, 이준석, 경북, 안동시, 안동댐, 물길, 로","{'메달': 5, '이재명': 5, '올림픽': 5, '코로나': 5, '도쿄올림픽..."
4,정치,_0809,안동서 토크콘서트 하는 이준석 대표,https://news.naver.com/main/read.naver?mode=LS...,"박홍식, 힘, 이준석, 경북, 안동시, 안동댐, 물길, 를, 로","{'메달': 5, '이재명': 5, '올림픽': 5, '코로나': 5, '도쿄올림픽..."
...,...,...,...,...,...,...
1841,IT/과학,_0809,[국감 IT 이슈④] 게임 ‘확률형 아이템’ 규제 재점화…셧다운제 향방은?,https://news.naver.com/main/read.naver?mode=LS...,"실효, 최소한, 유명무실, 셧, 다운, 제, 듯, 여가부, 초점, 마이크로소프트, ...","{'보안': 340, '글로벌': 99, '클라우드': 94, '중국': 80, '..."
1842,IT/과학,_0809,한국나노기술원 제7대 서광석 원장 취임,https://news.naver.com/main/read.naver?mode=LS...,"김현아, 광석, 나노기술, 나노기술, 서울대학교, 광석, 제, 광석, 서울대학교, ...","{'보안': 340, '글로벌': 99, '클라우드': 94, '중국': 80, '..."
1843,IT/과학,_0809,“태양 자기장의 근원은 자전 때 생기는 플라스마 소용돌이”,https://news.naver.com/main/read.naver?mode=LS...,"독일, 구진, 위성, 유럽우주국, 의, 태양, 탐사선, 솔, 오비터, 태양, 최, ...","{'보안': 340, '글로벌': 99, '클라우드': 94, '중국': 80, '..."
1844,IT/과학,_0809,인공지능 선장이 이끄는 ‘新 대항해시대’ 성큼,https://news.naver.com/main/read.naver?mode=LS...,"조선업, 후끈, 단, 항해, 개념, 장애물, 탐지, 등, 해상, 인지, 단, 중공업...","{'보안': 340, '글로벌': 99, '클라우드': 94, '중국': 80, '..."


# 최종 - TFIDF를 통한 주요기사 산출 및 클러스터링 작업

In [1]:
# 함수: 서로 다른 딕셔너리, 같은 key일때 value끼리 곱셈
def dict_mul(d1, d2):
    d3 = dict()
    for k in d1:
        if k in d2:
            d3[k] = d1[k] * d2[k]
    return d3

# 함수 :서로 다른 딕셔너리, 같은 key일때, A 딕셔너리기준으로 B딕셔너리의 같은 key의 value 탐색  
def dict_cod(d1,d2):
    d3 = dict()
    for k in d1:
        if k in d2:
            d3[k] = d1[k]
    return d3


tfidf_validation_data = []
count_validation_data = []
appended_data = []


for k in range(len(df_domain_count)):
    df_article_3[(df_article_3["DOMAIN"]==df_domain_count["DOMAIN"][k])]
    a = df_tfidf[(df_tfidf['DOMAIN']==df_domain_count['DOMAIN'][k])]
    a1 = df_tfidf_1[(df_tfidf_1['DOMAIN']==df_domain_count['DOMAIN'][k])]
    a2 = df_tfidf_2[(df_tfidf_2['DOMAIN']==df_domain_count['DOMAIN'][k])]
    a3 = df_tfidf_3[(df_tfidf_3['DOMAIN']==df_domain_count['DOMAIN'][k])]
    a4 = df_tfidf_4[(df_tfidf_4['DOMAIN']==df_domain_count['DOMAIN'][k])]
    #각 도메인별 tfidf 
    
    rawdata = pd.concat([a,a1,a2,a3,a4])
    rawdata = pd.DataFrame(rawdata.set_index(['DOMAIN']).reset_index()).drop(['TITLE','URL'],axis = 1)
    rawdata_list = list(rawdata.groupby(['DOMAIN', 'MM'], as_index=False).sum()['WORD_LIST'])
    
    tfidf = TfidfVectorizer(min_df = 1 ,max_df=3, sublinear_tf = True, ngram_range=(1,1),max_features=100
                           , stop_words = stop_word)
    features = tfidf.fit_transform(rawdata_list)
    df_tfidf_score = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())
    df_tfidf_score["sum"] = df_tfidf_score.sum(axis=1)
    df_tfidf_score_2 = df_tfidf_score.drop(['sum'],axis =1 ).tail(1)
    df_tfidf_score_2 = df_tfidf_score_2.reset_index().drop(['index'],axis =1) 
    
    #1 오늘자 tfidf feature과 score 추출 후 데이터프레임 조정
    df_tfidf_score_3 = df_tfidf_score_2.transpose() #좌우대칭
    df_tfidf_score_3 = df_tfidf_score_3[df_tfidf_score_3 != 0].dropna(axis=0)
    df_tfidf_score_3 = df_tfidf_score_3.sort_values(by=[0], ascending=False)
    df_tfidf_score_4 = df_tfidf_score_3.transpose() #좌우대칭
    df_tfidf_score_4=df_tfidf_score_4.reset_index()
    
   #2 최종 엑셀시트에 보여줄 금일자 tfidf관련 item형태의 데이터프레임
    df_tfidf_valid = df_tfidf_score_2.transpose()
    df_tfidf_valid=df_tfidf_valid.reset_index()
    df_tfidf_valid.columns = [0,1]
    cols = [0, 1]
    df_tfidf_valid[df_domain_count['DOMAIN'][k]] =df_tfidf_valid[cols].apply(lambda row: ' : '.join(row.values.astype(str)), axis=1)    
    df_tfidf_valid_2 = df_tfidf_valid.sort_values(by=1, ascending=False)
    df_tfidf_valid_2 = df_tfidf_valid_2[df_tfidf_valid_2 != 0].dropna(axis=0)
    df_tfidf_valid_2 = df_tfidf_valid_2.reset_index()
    df_tfidf_valid_3= df_tfidf_valid_2.drop(['index',0,1], axis=1)

    
    #3 최종 엑셀시트에 보여줄 금일자 count 관련 item형태의 데이터프레임
    df_domain_count_2 = df_domain_count.drop(['CUM_COUNT_WORD_TOP20'],axis=1)
    df_domain_count_2 = df_domain_count_2.set_index('DOMAIN')
    df_domain_count_3= df_domain_count_2['CUM_COUNT_WORD'].to_dict()
    df_domain_count_4 = df_domain_count_3[df_domain_count['DOMAIN'][k]]
    df_domain_count_5=pd.DataFrame(list(df_domain_count_4.items()),columns=['0', '1'])
    df_domain_count_5.columns = [0,1]
    cols = [0, 1]
    df_domain_count_5[df_domain_count['DOMAIN'][k]] =df_domain_count_5[cols].apply(lambda row: ' : '.join(row.values.astype(str)), axis=1)    
    df_domain_count_6= df_domain_count_5.drop([0,1], axis=1)
    
    

    critic_list = list(df_tfidf_score_4)
    
    
    #주요뉴스의 url과 제목 등을 보여줄 main 데이터프레임 
    df_main = df_article_5[(df_article_5['DOMAIN']==df_domain_count['DOMAIN'][k])]
    df_main = df_main.set_index(['DOMAIN']).reset_index()
    df_main_final = {'INDEX':[],'TFIDF_WORD_COUNT':[], 'TFIDF_SCORE':[], 'TFIDF_SET':[], 'TFIDF_LEN':[], 'TFIDF_TOTAL_SCORE':[]}
    
    for j in range(len(df_main)):
        result = [x for x in [s.replace(' ', '') for s in list(df_main['WORD_LIST'][j].split(','))] if x in critic_list]    
        dict_tfidf = df_tfidf_score_4.to_dict('records')[0]
        rc = dict(Counter(result))
        rcc =dict_mul(dict_tfidf,rc)
        rcc2 =dict_cod(dict_tfidf,rc)
        df_main_final['INDEX'].append(j)
        df_main_final['TFIDF_WORD_COUNT'].append(rc)
        df_main_final['TFIDF_SCORE'].append(rcc2)
        df_main_final['TFIDF_SET'].append(str(list(dict(rcc).keys())))
        df_main_final['TFIDF_LEN'].append(len(rc))
        df_main_final['TFIDF_TOTAL_SCORE'].append(sum(rcc2.values())*len(rc))
    df_main_final = pd.DataFrame(df_main_final)
    
    df_main_final_2 = pd.concat([df_main, df_main_final], axis=1)
    df_main_final_2 = df_main_final_2.sort_values(by=["TFIDF_TOTAL_SCORE"], ascending=False)
    df_main_final_2 = df_main_final_2[df_main_final_2 != 0].dropna(axis=0)
    
    
    # 군집화할 대상
    content = df_main_final['TFIDF_SET'].tolist()
    
    
    # 군집화 할 그룹의 갯수 정의
    n_clusters = 8
    
    # CountVectrizer로 토큰화
    vectorizer = CountVectorizer(stop_words = stop_word)
    X = vectorizer.fit_transform(content)

    # l2 정규화
    X = normalize(X)

    # k-means 알고리즘 적용
    kmeans = KMeans(n_clusters = n_clusters,max_iter=10000, init='k-means++').fit(X)

    # trained labels and cluster centers
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    df_main_final['labels'] = labels
    
    df_main_final_3 = pd.concat([df_main, df_main_final], axis=1)
    df_main_final_3 = df_main_final_3.sort_values(by=["TFIDF_TOTAL_SCORE"], ascending=False).dropna(axis=0)
    df_main_final_3 = df_main_final_3[df_main_final_3 != 0].dropna(axis=0)
    df_main_final_3 = df_main_final_3.loc[df_main_final_3.groupby(["labels"])["TFIDF_TOTAL_SCORE"].idxmax()]
    df_main_final_4 = df_main_final_3.loc[df_main_final_3.groupby(["TFIDF_SET"])["TFIDF_TOTAL_SCORE"].idxmax()]
    df_main_final_4 = df_main_final_4.sort_values(by=["TFIDF_TOTAL_SCORE"], ascending=False)
    appended_data.append(df_main_final_4)
    tfidf_validation_data.append(df_tfidf_valid_3)
    count_validation_data.append(df_domain_count_6)
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
appended_data = appended_data[(appended_data['DOMAIN'] != '생활/문화')] 
tfidf_validation_data = pd.concat(tfidf_validation_data,axis=1)
count_validation_data = pd.concat(count_validation_data,axis=1)

final_data = appended_data[['DOMAIN','SUB_DOMAIN','PRESS','TITLE','URL']]
domain_tfidf = appended_data


writer = pd.ExcelWriter(base_dir+'naver_news_data'+mm+'.xlsx', engine='xlsxwriter')


final_data.to_excel(writer,sheet_name='최종')
count_validation_data.to_excel(writer,sheet_name='도메인별_최빈단어')
domain_tfidf.to_excel(writer,sheet_name='TF_IDF_산출근거')
tfidf_validation_data.to_excel(writer,sheet_name='도메인별_TF_IDF')

writer.save()
writer.close()


    

    

NameError: name 'df_domain_count' is not defined